# Text Mining met Python - OVV Rapporten

|   |   |
|------|------|
|__Auteur__ | Marlon van den Hoek  |
| __E-mail__  | <m.vandenhoek@onderzoeksraad.nl>  |
| __Onderwerpen__ | text mining, cleansing, tokenizing, stemming, stopwords, TF-IDF, Jaccard similarity, KMeans, hierarchical document clustering, LDA, sentiment analysis |
| __Afhankelijkheden__ | `nltk`, `textmining`, `pdfminer`, `re`, `multiprocessing`, `sklearn`, `pandas`, `wordcloud`, `gensim`, `plotly` (alleen voor plotting) |
|__Datum__ | 14-02-2018 |
| __Versie__ | `0.7` |

_Opmerking 1_: Om dit Jupyter Notebook correct te kunnen runnen moeten eerst de juiste pakketten (zie afhankelijkheden) geinstalleerd worden. Dit kan bijvoorbeeld met `pip`. Zie hiervoor de eerste cell hier onder.

_Opmerking 2_: Dit notebook is standaard beperkt tot 30 documenten. Om alle documenten te analyseren, verwijder de limiet van 30 bij het laden: verander `arange` bij de for loop, `job` bij de multicore loop.

In [ ]:
# To install the dependencies, run the following code -- run only once!!
! pip install nltk
! pip install textmining
! pip install pdfminer
! pip install multiprocessing
! pip install sklearn
! pip install gensim
! pip install plotly
! pip install wordcloud

## Introductie

In dit Jupyter notebook gaan we aan de slag met text mining op de (openbare) rapporten van de Onderzoeksraad. Met een ander script (scraper) zijn meer dan 600 rapporten van de OVV website gedownload. Daarin is rekening gehouden met andere documenten zoals errata, bijlagen, aanbevelingen; deze documenten zijn voor het grootste gedeelte uit de collectie gefilterd. Deze documenten zijn opgeslagen in `./Rapporten/`.

Omdat de meeste rapporten van de OVV zijn voorzien van een beveiliging tegen het kopieren van tekst, zijn alle rapporten eerst gedecrypt met een extern programma. De gedecrypte bestanden zijn te vinden in de map `./Rapporten decrypted/`. De onderliggende submappen geven de taal van het document aan: Nederlands `nl` of Engels `en`.

Er is een handmatige selectie van rapporten gemaakt en geplaatst in `./Rapporten selectie/nl` met een validatie set in `./Rapporten selectie/validatie`.

#### Mappenstructuur
1. Rapporten
    - en: bevat gedownloade Engelse rapporten.
    - nl: bevat gedownloade Nederlandse rapporten.
2. Rapporten decrypted
    - en: bevat gedecrypte Engelse rapporten.
    - nl: bevat gedecrypte Nederlandse rapporten.
3. Rapporten selectie
    - en: bevat Engelse rapporten.
    - nl: bevat Nederlandse rapporten.
    - validatie: bevat documenten ter validatie. Gebruiker moet zelf nl of en documenten als validatieset selecteren en plaatsen in deze map.
4. Output
    - logfile
    - plots


#### NLTK en overige pakketten
Voor het minen maken we gebruik van de natural language toolkit ([NLTK](http://www.nltk.org/)), het textmining pakket ([textmining1.0](https://pypi.python.org/pypi/textmining/1.0)) en het boek _Natural Language Processing with Python_<sup id="a1">[1](#f1)</sup>. Daarnaast maken we ook gebruik van pakketten voor lineaire algebra en matrix operaties, e.a. Deze pakketten importeren we eerst.

<b id="f1">1</b> Bird, S., Klein, E., & Loper, E. (2009). Natural language processing with Python: analyzing text with the natural language toolkit. "O'Reilly Media, Inc.". [↩](#a1)

In [ ]:
# import dependencies
import numpy as np
import nltk, textmining
import sys
import os
import datetime # to set date and time
from cStringIO import StringIO
import scipy as sc
import matplotlib.pyplot as plt
import re
import pandas as pd

## Vorige sessie laden

Indien we een corpus hebben opgebouwd in een vorige sessie kunnen we deze hier laden. In het bestand moeten de volgende variabelen zijn opgeslagen:
1. lang
2. FilePath
3. FileNames
4. run_settings
5. textdict
6. metadict

Als je van plan bent om de teksten opnieuw te processen en tokenizen, sla deze stap over!

Laad de variabelen als volgt:


In [ ]:
lang = str(np.load('./Vars/lang.npy'))
FilePath = str(np.load('./Vars/filepath.npy'))
FileNames = np.load('./Vars/filenames.npy')
run_settings = eval(str((np.load('./Vars/run_settings.npy'))))
textdict = eval(str((np.load('./Vars/textdict.npy'))))
metadict = eval(str((np.load('./Vars/metadict.npy'))))

# set date to current date
#now = datetime.datetime.now()
#run_settings['datetime'] = now.strftime("%d-%m-%Y %H:%M")

print(run_settings)
print('Total number of tokens: ',np.shape(sum(list(textdict.values()),[]))[0])

## Nieuwe text laden

Als we gebruik willen maken van onze eigen corpus, database met text, dan moeten we de rapporten die gedownload zijn in pdf eerst converteren naar plain text (txt). Hiervoor gebruiken we `pdfminer`. Metadata halen we uit de pdf bestanden met het pakket `pyPdf` omdat deze sneller is.

In de volgende code importeren we eerst alle pakketten die nodig zijn, daarna definieren we de bestandlocatie en creeeren we een functie om een bestand te kunnen lezen. In deze functie wordt elke pagina van het desbetreffende bestand geparsed en bij elkaar gevoegd.

Om bij te houden welke acties er zijn uitgevoerd, welke settings zijn gebruikt en de datum en tijd van de run maken we ook nog een settings dict die zal worden geprint bij het uitvoeren van log files.

In [ ]:
# import packages
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from pyPdf import PdfFileReader # extract pdf metadata

# Define language
lang = 'nl'

# Define location of reports
#FilePath = './Rapporten decrypted/' + lang + '/'
FilePath = './Rapporten selectie/' + lang + '/'

# Get files in folder
FileNames = os.listdir(FilePath)

# Make function to convert pdf to text
def pdfparser(FilePath,FileName): 
    fp = file(FilePath+FileName, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # Process each page contained in the document.
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    
    # return text
    return data

In [ ]:
# define settings dict, will be updated when running certain components
now = datetime.datetime.now()
run_settings = {"title": "OVV Text Mining - Rapporten", "datetime": now.strftime("%d-%m-%Y %H:%M"), 
                "text_stemmed": False, "stopwords_removed": False, "language": lang, "path": FilePath,
                "dendogram": None, "kmeans": None, "kmeans_html": None, "timeline": None, "kmeans_pca": None,
                "kmeans_pca_html": None, "kmeans_tsne": None, "kmeans_tsne_html": None}

In de volgende cell definieren we een functie om te text te processen en tokenizen. Deze functie is van belang voor een aantal routines in de rest van dit document, het is daarom belangrijk deze cell te runnen.
<a name="processtokenize"></a>

In [ ]:
def processandtokenize(FileName, dictname, metadict): 
    
    # print progress, var FileNames is taken from shared memory, not required as function input
    print "Processing: ",int(np.arange(0,len(FileNames))[np.array(FileNames) == FileName])+1,'/',len(FileNames), FileName
    
    try:
        data = pdfparser(FilePath,FileName)
        
        # get PDF metadata
        pdfi = PdfFileReader(open("".join([FilePath,FileName]), "rb"))
        pdfi = pdfi.getDocumentInfo()
        metadict[FileName] = pdfi

        # split text
        data = np.array(re.split('\W+',data),dtype=str) # tokenize text
        
        # convert all data to lower case
        data = np.array([w.lower() for w in data])

        ## filter out empty cells
        ind = data != ''
        data = data[ind]

        ## filter out non-alpha strings
        ind = np.char.isalpha(data)
        data = data[ind]

        if data.size is not 0:
            # filter out strings with length = 1
            strlen = np.vectorize(len)
            ind = strlen(data) != 1
            data = data[ind]
        
        # add data to dict
        dictname[FileName] = list(data)

    except ValueError:
        print "Skipping ", FileName

Nu de functie om een bestand te lezen gedefineerd is kunnen we deze functie toe gaan passen op alle bestanden. We doen dit door over de bestanden heen te loopen.

In de loop gebeurt het volgende: 
1. het bestand wordt gelezen
2. de metadata wordt gelezen en opgeslagen in de dict `metadict`
3. met het Regexp `re` pakket splitten we de tekst
4. converteren de tekst naar lower case
5. uitfilteren van lege indices
6. uitfilteren van numerieke waarden
7. als de overgebleven body niet `None` is dan voegen we de tokens toe aan dict `textdict`

_**Let op**_ om geparallelliseerd bestanden te lezen en converteren, sla de volgende stappen over en ga naar [hier](#parallel). Als je systeem voorzien is van meerdere cores kan dit significant sneller zijn!

In [ ]:
# Prealloceer variabelen
textdict = {}
metadict = {}

for FileName in FileNames:
    processandtokenize(FileName, textdict, metadict)

### Geparallelliseerd bestanden lezen en converteren
 <a name="parallel"></a>
 
Om een groot gedeelte van de `for` loops sneller te maken, kunnen we de processen parallelliseren met `multiprocessing`. Dit pakket moeten we eerst importeren en dan definieren wat we ermee willen doen.

Om het uitlezen van bestanden te kunnen faciliteren maken we gebruik van de functie _processandtokenize_. Deze functie wordt parallel uitgevoerd en de output is een dict met daarin al onze bestandnamen en bijbehorende tokens.

_Let op_: het runnen van onderstaande code is erg CPU intensief en kan ervoor zorgen dat je systeem langzaam wordt gedurende het processen van de data.

In [ ]:
import multiprocessing      

# initialize multi-core process
#manager = multiprocessing.Manager()
#textdict = manager.dict()
#metadict = manager.dict()

# define job
#job = [multiprocessing.Process(target=processandtokenize, args=(i, textdict, metadict)) for i in FileNames] # for all files


# start all jobs in the job list
#_ = [p.start() for p in job]
#_ = [p.join() for p in job]

# convert the multiprocess textdict opject to normal dict
#textdict = dict(textdict)
#metadict = dict(metadict)

if __name__ == "__main__":
    
    # initialize manager and shared memory vars
    manager = multiprocessing.Manager()
    num_parprocess = multiprocessing.cpu_count()-1
    textdict = manager.dict()
    metadict = manager.dict()
    
    # initialize pool and add processes
    pool = multiprocessing.Pool(processes=num_parprocess)
    for i in FileNames:
        pool.apply_async(processandtokenize, args=(i, textdict, metadict))
    pool.close()
    pool.join()

# convert shared vars to normal dict
textdict = dict(textdict)
metadict = dict(metadict)

In [ ]:
print 'Total number of tokens: ',np.shape(sum(list(textdict.values()),[]))[0]


## Stopwords verwijderen

Om onze teksten te processen moeten we nog een cleansing stap uitvoeren: het verwijderen van de stopwords. Hiervoor gebruiken we de standaard NL/EN library in de NLTK toolkit. Om een overzicht te krijgen van de stopwords in het NLTK pakket, gebruik het commando in de tweede cell. Het is mogelijk om hier meer woorden aan toe te voegen.

In [ ]:
# Import required packages
from nltk.corpus import stopwords
from nltk import ngrams
from collections import Counter

useownstopwords = True
stopwords_path = './Stopwords/'

if useownstopwords:
    sw_dutch = pd.Series.from_csv(stopwords_path + 'stopwords_nl.csv', index_col=False).tolist()
    sw_english = pd.Series.from_csv(stopwords_path + 'stopwords_en.csv', index_col=False).tolist()
else:
    sw_dutch = stopwords.words('dutch')
    sw_english = stopwords.words('english')

# define function to remove stopwords
def removestopwords(textdict):
    for i in np.arange(0,np.shape(textdict.keys())[0]):
        print i+1,'/',np.shape(textdict.keys())[0]
        if lang == 'nl':
            text_nostopw = [w for w in textdict.values()[i] if not w in sw_dutch and
                                                               not w in sw_english] # haal voor zekerheid ook engelse eruit
        elif lang == 'en':
            text_nostopw = [w for w in textdict.values()[i] if not w in sw_english]
        textdict[textdict.keys()[i]] = text_nostopw

In [ ]:
# remove stopwords
removestopwords(textdict)
    
# update settings dict
run_settings['stopwords_removed'] = True

# trick to flatten matrix: sum(list(textdict.values()),[])
count = Counter(sum(list(textdict.values()),[]))

# print de meest voorkomende woorden na het verwijderen van de stopwords
print(count.most_common(100))

## Wordcloud visualisatie

Voordat we verdergaan met het stemmen van de woorden, kunnen we een visualisatie maken van de top 300 woorden met de volgende code:


In [ ]:
from wordcloud import WordCloud
from collections import Counter

count = Counter(sum(list(textdict.values()),[]))
words = count.most_common(400)
wcloud = []

for word, num in words:
    wcloud = wcloud + [word] * num

wcloud = " ".join(wcloud)

wordcloud = WordCloud(background_color='white',stopwords=None,
                      max_font_size=70,scale=3,random_state=1, 
                      collocations=False,width=1200, 
                      height=720, max_words=400).generate(wcloud)

plt.figure(figsize=(40, 24))
plt.axis('off')
plt.imshow(wordcloud)
plt.savefig('./Output/Rapporten - wordcloud.png')
plt.show()


## Text stemming

Nu alle woorden uit de text gehaald zijn en in een `numpy` array staan kunnen we de woorden gaan _stemmen_. We gebruiken hiervoor de Porter stemmer voor het Engels en de Snowball stemmer in de NLTK toolkit voor het Nederlands.

Let op: de huidige implementatie van de Snowball stemmer is verre van perfect voor de Nederlandse taal. Deze stap kan ook overgeslagen worden.

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

print "Supported Languages Snowball: \n"
print(" ".join(SnowballStemmer.languages))
print "\n"

print "Supported Languages Porter: \n"
print "english\n"

# define function to stem text
def stemtext(textdict):
    if lang == 'nl':
        stemmer = SnowballStemmer("dutch")
    elif lang == 'en':
        stemmer = PorterStemmer()

    for i in np.arange(0,np.shape(textdict.keys())[0]):
        text_stemmed = []
        print i+1,'/',np.shape(textdict.keys())[0]
        for j in np.arange(0,np.shape(textdict.values()[i])[0]):
            text_stemmed.append(stemmer.stem(textdict.values()[i][j]))
        textdict[textdict.keys()[i]] = text_stemmed

In [ ]:
# stem text
print "Stemming text: \n"
stemtext(textdict)

# update settings struct
run_settings['text_stemmed'] = True


## Jaccard Similarity

Om verschillende documenten met elkaar te vergelijken maken we gebruik van de _Jaccard Similarity_: $J\left(A,B\right) = \frac{|A\bigcap B|}{|A\bigcup B|}$. Dit is de lengte van de intersectie (overeenkomst) van de set tokens van tekst $A$ en tekst $B$ gedeeld door de union (hele collectie) van beide sets. Deze measure geeft de numerieke overeenkomst, op een schaal van 0-1, tussen twee documenten. Met andere woorden: de Jaccard Similarity is een fractie die aangeeft hoeveel van tekst $A$ en $B$ overeenkomen. Set $A$ en set $B$ hoeven niet per definitie de zelfde grootte of lengte te hebben.

![Image](http://4.bp.blogspot.com/-x1n8llkCjuc/UAyeWdyc3EI/AAAAAAAAAU8/ron4CjbDVDM/s1600/DATA+SETS.png)

In [ ]:
# define Jaccard similarity
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return float(len(intersection))/len(union)

# calculate the Jaccard similarity of text 1 and text 2
A = textdict.values()[0]
B = textdict.values()[1]
print jaccard_similarity(A,B)


De Jaccard Similarity kan op veel verschillende manieren worden berekend. We kunnen de measure berekenen voor:
1. de bag of words zonder rekening te houden met woordfrequentie, we gebruiken dan alleen de set met unieke woorden voor beide teksten
2. eerst de stopwords verwijderen en dan de bag of words gebruiken zonder frequentie
3. rekening houden met de woordfrequentie
4. meer geavanceerde methodes zoals k-shingles of n-grams

In deze implementatie is ervoor gekozen om 2. te gebruiken (afhankelijk van de eerder uitgevoerde stappen kan ook 1. gebruikt worden).

Om de intersectie van beide teksten te bekijken, gebruik het volgende commando:

In [ ]:
# to check the intersection of both texts:
set(A).intersection(set(B))

## TF-IDF - _term frequency-inverse document frequency_

Om het belang van verschillende woorden in een corpus numeriek uit te drukken berekenen we de TF-IDF. Dit doen we door gebruik te maken van de functie `feature_extraction` in de `sklearn` toolbox (onderdeel van `scikit`). 

De TF-IDF measure reflecteert hoe belangrijk een woord voor een document is door in eerste instantie te kijken hoe vaak een woord voor komt in een tekst. In het meest simpele geval gebruiken we hiervoor de woordfrequentie $f_{t,d}$. Ons algorimte maakt gebruik van de geaugmenteerde woordfrequentie $tf$:

$\text{tf}(t,d) = 0.5 + \frac{0.5 ~\times ~f(t,d)}{\text{max}\left\{ f(w,d):~w~\in ~d\right\}}$

waarin de noemer de maximale woordfrequentie aangeeft.

Omdat sommige woorden vaak in tekst aanwezig zijn maar eigenlijk weinig betekenis hebben maken we gebruik van de inverse document frequentie om de uitkomst van de woord frequentie te reguleren. De IDF is gedefinieerd als:

$ \text{idf}(t) = \log \frac{n_d}{df\left( d,t \right)} + 1$

hierin is $n_d$ het totaal aantal documenten en $df\left(t,d\right)$ het aantal documenten dat term $t$ bevat.

De TF-IDF krijgen we dan door:

$ \text{tf-idf}(t,d) = \text{tf}(t,d) \times \text{idf}(t)$

De TF-IDF wordt berekend voor alle unieke termen $t$ in alle unieke documenten $d$. De resulterende matrix heeft dus de dimensie $\left( t,d \right)^{\intercal}$ waarin element ($t_i,d_j)^{\intercal}$ de numerieke relevantie geeft van term $i$ in document $j$.

![Image of Yaktocat](http://www.jiem.org/index.php/jiem/article/viewFile/293/252/2402)

#### Limitatie bag of words
Omdat een simpel bag of words model geen rekening houdt met verkeerd gespelde woorden/typo's kunnen deze woorden als verschillend worden gezien door het algoritme. Om dit te voorkomen kunnen we gebruik maken van een collectie bigrams (n=2) of ngrams (n>2). Met een analyzer (in dit geval `char_wb`) en een gedefinieerde `ngram_range` kunnen we woorden dan herkennen aan $n$ features. Zo kunnen we voorkomen dat simpele fouten invloed hebben op het model.

**Let op**: hoe meer ngrams features, hoe meer geheugen nodig is!

#### Tokenizer definitie<a name="limitatie"></a>
Omdat `Tfidfvectorizer` alleen de volledige tekst als string accepteert moeten we eerst de tokens in `textdict` manipuleren -> we voegen de tokens samen tot een string en splitten die dan weer op de spatie tussen woorden. Dit is een workaround en genereert een kleine overhead.

#### Overige parameters
De standaardimplementatie van de vectorizer in de `sklearn` toolkit heeft een aantal parameters die aangepast kunnen worden, waaronder:
1. __max_df__: dit is de maximale frequentie die een feature in documenten kan hebben om opgenomen te worden in de TF-IDF matrix. Features (woorden) met een hogere frequentie worden genegeerd. Als een term bijvoorbeeld in meer dan 80% van de documenten voorkomt heeft het waarschijnlijk weinig betekenis en/of is het lastig om aan de hand van deze feature documenten te onderscheiden.
2. __min_idf__: minimale woordfrequentie die aangeeft in hoeveel documenten ten opzichte van de totale collectie een bepaald woord voorkomt, features met een lagere frequentie dan min_idf worden genegeerd.
3. __ngram_range__: zie [limitatie bag of words en ngrams](#limitatie), de ngram_range (a,b) geeft aan dat alle mogelijke combinaties van woorden tussen lengte a en b gebruikt kunnen worden als feature als deze binnen de gestelde min_idf en max_idf vallen. Dit voorkomt dat verkeerde gespelde woorden, vervoegingen, etc. invloed uitoefenen op de resultaten.
4. __use_idf__: gebruik inverse-document-frequentie
5. __smooth_idf__: smooth idf door 1 bij de document frequentie op te tellen, dit voorkomt delen door nul en Inf errors.
6. __sublinear_tf__: gebruik sublineaire schaling, vervang $tf$ door $1+\log(tf)$.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# manipulate variable textdict to allow feeding into TfidfVectorizer function
alltext = [' '.join(text) for text in textdict.values()]

# define tokenizer
# dummy function: we split concatenated tokens at space
tfidf_tokenizer = lambda doc: [word for word in doc.split(' ')]

# define range of ngrams
ngrams = (3,15)

# define vectorizer -- using ngrams
sklearn_tfidf = TfidfVectorizer(analyzer='char_wb',ngram_range=ngrams,norm='l2',min_df=1,max_df=0.8,
                                use_idf=True,smooth_idf=False, sublinear_tf=True, tokenizer=tfidf_tokenizer)

# define vectorizer -- unigram
#sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0,max_df=0.8,use_idf=True,smooth_idf=False, 
#                                sublinear_tf=True, tokenizer=tfidf_tokenizer)

# calculate TFIDF matrix
tfidf_matrix = sklearn_tfidf.fit_transform(alltext)

# calculate distance
dist = 1 - cosine_similarity(tfidf_matrix)

#print tfidf_matrix.toarray()[0].tolist()
terms = sklearn_tfidf.get_feature_names()

print(np.shape(tfidf_matrix))

In [ ]:
# export tfidf matrix to csv
#import pandas as pd

#index = textdict.keys()
#cols = sklearn_tfidf.get_feature_names()

#df = pd.DataFrame(tfidf_matrix.todense())

#df.to_csv('data.csv')

Print de lijst met termen. Let op, bij grote ngram_range is dit een zeer lange lijst.

In [ ]:
if np.shape(tfidf_matrix)[1] < 50000:
    print terms
else:
    print 'List too long to print'

## Clustering en dimensiereductie

Aan de hand van de TF-IDF matrix kunnen we de cosine similarity uitrekenen en de onderlinge (numerieke) afstand tussen verschillende text bestanden. Op basis van deze measure kan er dan met het KMeans algoritme een onderscheid gemaakt worden tussen verschillende clusters.

Het aantal clusters moeten we zelf definieren. Dit is onderhevig aan trial en error: het zoeken van het optimale aantal clusters is geen triviaal proces en wordt vaak aangepakt met een brute force aanpak. Dit komt dus neer op $m$ keer het algoritme zijn clusters laten bepalen voor $n$ verschillende clustergrootten en achteraf bepalen met welke clustergrootte het beste resultaat wordt bereikt.

KMeans clusters door documenten zo te groeperen waardoor de variantie (statistische spreiding) van de groep minimaal is. Dit kan op twee manieren benaderd worden:

1. KMeans direct toepassen op een $n$-dimensionale dataset. Dit verdeelt de set met documenten over clusters maar visualisatie is niet direct mogelijk.
2. De dataset eerst reduceren naar het tweedimensionale vlak met PCA (mathematisch) en vervolgens KMeans toepassen. Dit staat wel toe om de clusters grafisch te representeren.

Naast de bovengenoemde benaderingen passen we ook nog een probabilistische dimensiereductiemethode toe:
3. De dataset eerst reduceren naar tweedimensionaal vlak met t-SNE (probabilistisch) en vervolgens KMeans toepassen.

Voor alle clusteringmethodes gebruiken we de volgende settings:

In [ ]:
num_clusters = 11 # number of clusters
max_iter = 10000 # maximum number of iterations

### KMeans op $n$-dimensionale dataset

We passen eerst KMeans direct toe op de $n$-dimensionale dataset.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

name = "kmeans"
run_settings[name] = True

km = KMeans(n_clusters=num_clusters, max_iter=10000)
km.fit(tfidf_matrix)

# print clusters
for i in np.arange(0,num_clusters):
    print "Cluster", i, ": \n"
    print np.array(textdict.keys())[np.array(km.labels_.tolist()) == i]
    print "\n\n"
    
clusters = km.labels_.tolist()    

In [ ]:
# output clusters to text file
with open("./Output/Run logfile "+run_settings["datetime"]+".txt", "a") as text_file:
    text_file.write("====================================\n")
    text_file.write("         K-MEANS CLUSTERING         \n")
    text_file.write("====================================\n\n")
    for i in np.arange(0,num_clusters):
        text_file.write("Cluster {}\n".format(i))
        text_file.write("{}".format(np.array(textdict.keys())[np.array(km.labels_.tolist()) == i]))
        text_file.write("\n\n")

### Eerst dimensiereductie PCA dan KMeans

Alvorens te clusteren met het KMeans algoritme reduceren we het aantal dimensies eerst naar $n=2$ met behulp van PCA.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# set name
name = 'kmeans_pca'
run_settings[name] = True

# first reduce dimensions to two
pca = PCA(n_components=2).fit(tfidf_matrix.todense())
X = pca.transform(tfidf_matrix.todense()) # transformed data

km = KMeans(n_clusters=num_clusters, max_iter=max_iter)
km.fit(X)

# print clusters
for i in np.arange(0,num_clusters):
    print "Cluster", i, ": \n"
    print np.array(textdict.keys())[np.array(km.labels_.tolist()) == i]
    print "\n\n"
    
clusters = km.labels_.tolist()

In [ ]:
# output clusters to text file
with open("./Output/Run logfile "+run_settings["datetime"]+".txt", "a") as text_file:
    text_file.write("====================================\n")
    text_file.write("       K-MEANS PCA CLUSTERING       \n")
    text_file.write("====================================\n\n")
    for i in np.arange(0,num_clusters):
        text_file.write("Cluster {}\n".format(i))
        text_file.write("{}".format(np.array(textdict.keys())[np.array(km.labels_.tolist()) == i]))
        text_file.write("\n\n")

### Eerst dimensiereductie t-SNE daarna KMeans

Alvorens te clusteren met het KMeans algoritme reduceren we het aantal dimensies eerst naar $n=2$ met behulp van t-SNE. t-SNE is, in tegenstelling tot PCA (mathematisch), een probabilistische methode om de dimensies te reduceren.

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

# set name
name = 'kmeans_tsne'
run_settings[name] = True

# first reduce dimensions to two
tsne = TSNE(n_components=2, verbose=1, perplexity=20, n_iter=10000)
X = tsne.fit_transform(tfidf_matrix.todense()) # transformed data

km = KMeans(n_clusters=num_clusters, max_iter=max_iter)
km.fit(X)

# print clusters
for i in np.arange(0,num_clusters):
    print "Cluster", i, ": \n"
    print np.array(textdict.keys())[np.array(km.labels_.tolist()) == i]
    print "\n\n"
    
clusters = km.labels_.tolist()

In [ ]:
# output clusters to text file
with open("./Output/Run logfile "+run_settings["datetime"]+".txt", "a") as text_file:
    text_file.write("====================================\n")
    text_file.write("      K-MEANS t-SNE CLUSTERING      \n")
    text_file.write("====================================\n\n")
    for i in np.arange(0,num_clusters):
        text_file.write("Cluster {}\n".format(i))
        text_file.write("{}".format(np.array(textdict.keys())[np.array(km.labels_.tolist()) == i]))
        text_file.write("\n\n")

### Cluster visualisatie

Gebruik de onderstaande scripts om per clustermethode (na dimensiereductie) de clusters te visualiseren. `X` is de set getransformeerde data.

In [ ]:
# visualize cluster boundaries

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = np.max([np.abs(X[:,0].min()), np.abs(X[:,0].max())])/200   

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = X[:, 0].min(), X[:, 0].max()
y_min, y_max = X[:, 1].min(), X[:, 1].max()
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = km.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(16,12))
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(X[:, 0], X[:, 1], 'k.', markersize=20)

# Plot the centroids as a white X
centroids = km.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=100, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the dataset (' + name + ')' + '\n'
          'Centroids are marked with white cross', fontsize=16)
plt.xlim([np.min(X[:, 0]),np.max(X[:, 0])])
plt.ylim([np.min(X[:, 1]),np.max(X[:, 1])])
plt.xticks(())
plt.yticks(())

# save figure
plt.savefig('./Output/Rapporten-boundaries-' + name + ' ' + run_settings['datetime'] + '.pdf')
run_settings[name] = './Output/Rapporten-boundaries-' + name + ' ' + run_settings['datetime'] + '.pdf'

# plot figure
plt.show()
    

In [ ]:
# visualize clusters
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools

# get 2-dimensional data
xs, ys = X[:, 0], X[:, 1]

# visualize clusters
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=textdict.keys())) 
groups = df.groupby('label')

fig, ax = plt.subplots(figsize=(12, 8)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

cluster_names = ['Cluster ' + str(w) for w in np.arange(np.min(clusters)+1,np.max(clusters)+2,1)]
np.random.RandomState = 0
cluster_colors = plt.cm.jet(np.linspace(0,0.9,num_clusters))


for cname, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[cname], color=cluster_colors[cname], 
            mec='none')
    ax.set_aspect('auto')
    
# plot centroid numbers
centroids = km.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1], marker='o',
                c="white", alpha=0.2, s=400, edgecolor='k')
for i in np.arange(0,num_clusters):
    plt.text(centroids[i, 0]-0.006, centroids[i, 1]-0.015, str(i+1))
    
plt.title('K-means clustering on the dataset (' + name + ')' + '\n',
          fontsize=16)

# convert plot object to plotly object
fig = tools.mpl_to_plotly(fig)

options = {"displayModeBar": True,"modeBarButtonsToRemove": ['sendDataToCloud'],"showLink": False,
           "displaylogo": False, "setBackground": "transparent"} 

# set legend and layout options
fig['layout']['showlegend'] = True
fig['layout']['xaxis1']['ticks'] = ''
fig['layout']['xaxis1']['showticklabels'] = False
fig['layout']['yaxis1']['ticks'] = ''
fig['layout']['yaxis1']['showticklabels'] = False
fig['data'][-1]['name'] = 'Centroids'
fig['data'][-1]['text'] = 'Cluster center'

# set plot hover text
for i in np.arange(0,num_clusters):
    fig['data'][i]['text'] = np.array(textdict.keys())[np.array(km.labels_.tolist()) == i].tolist()

# plot and embed in ipython notebook
py.iplot(fig, config=options)

# save to file, uncomment to save
run_settings[name] = './Output/Rapporten-clusters-' + name + ' ' + run_settings['datetime'] + '.html'
py.plot(fig, config=options, filename='./Output/Rapporten-clusters-' + name + ' ' + run_settings['datetime'] + '.html')

### Clusteroptimalisatie

Bij de bovenstaande clustermethodes is gebruik gemaakt van een vooraf bepaald aantal clusters. Clusteroptimalisatie, het bepalen van het optimale aantal clusters, is nog een van de openstaande vraagstukken in de statistiek. Er zijn verschillende methodes om het optimale aantal clusters te bepalen. Wij maken hier gebruik van silhouetteanalyse.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# Create a subplot with 1 row and 2 columns
fig, axs = plt.subplots(np.shape(range_n_clusters)[0], 2)
fig.set_size_inches(22, 15*np.shape(range_n_clusters)[0])

for n_clusters in range_n_clusters:
    
    ind = sorted(range_n_clusters).index(n_clusters)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    axs[ind][0].set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    axs[ind][0].set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    #print("For n_clusters =", n_clusters,
    #      "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.spectral(float(i) / n_clusters)
        axs[ind][0].fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        axs[ind][0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    axs[ind][0].set_title("Silhouette analysis for KMeans ({}) clustering on sample data"
                  "with n_clusters = {}".format(name,n_clusters))
    axs[ind][0].set_xlabel("The silhouette coefficient values")
    axs[ind][0].set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    axs[ind][0].axvline(x=silhouette_avg, color="red", linestyle="--")

    axs[ind][0].set_yticks([])  # Clear the yaxis labels / ticks
    axs[ind][0].set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.spectral(cluster_labels.astype(float) / n_clusters)
    axs[ind][1].scatter(X[:, 0], X[:, 1], marker='.', s=50, lw=0, alpha=0.7,
                c=colors, edgecolor='k')
    
    #add label in x,y position with the label as the film title
    for j in range(len(df)):
        axs[ind][1].text(X[j, 0]*1.05, X[j, 1], df.iloc[j]['title'], size=6)  

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    axs[ind][1].scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        axs[ind][1].scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    axs[ind][1].set_title("The visualization of the clustered data.")
    axs[ind][1].set_xlabel("Feature space for the 1st feature")
    axs[ind][1].set_ylabel("Feature space for the 2nd feature")

    #plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
    #              "with n_clusters = %d" % n_clusters),
    #             fontsize=14, fontweight='bold')
    
fig.savefig('./Output/Rapport-clusteranalysis' + name + '.pdf')

plt.show()

### Hierarchical document clustering


In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram, linkage
import matplotlib.pyplot as plt
%matplotlib inline

dist = 1 - cosine_similarity(tfidf_matrix)
linkage_matrix = linkage(dist, 'ward') # ward/single/average/complete

fig, ax = plt.subplots(figsize=(15, 15./60*len(textdict.keys())))

ax = dendrogram(linkage_matrix, orientation="right", labels=textdict.keys());

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout
plt.savefig('./Output/Rapport-dendogram_plot ' + run_settings['datetime'] + '.pdf')
run_settings['dendogram'] = './Output/Rapport-dendogram_plot ' + run_settings['datetime'] + '.pdf'

## Latent Dirichlet Allocation - Topic Modeling

Om meer van de verborgen structuur in de tekstdocumenten te vinden kunnen we gebruik maken van _Latend Dirichlet Allocation_. LDA is een propabilistisch topic model die aanneemt dat documenten een mix zijn van verschillende onderwerpen (topics) en dat elk woord in het document bijdraagt aan dit onderwerp (en zo komt het onderwerp tot stand).

![Image](http://mengjunxie.github.io/ae-lda/img/IntroToLDA.png)

Om LDA te gebruiken moeten we eerst een model trainen met de data (teksten) die we hebben.

#### LDA model parameters
Voor het trainen van een LDA model gebruiken we de standaard implementatie van `gensim`. In dit model zijn een aantal aan te passen parameters:

1. __no_below__: houd tokens die minstens in dit aantal documenten aanwezig zijn.
2. __no_above__: verwijder tokens die in dit aantal (fractie van totaal) documenten aanwezig zijn.
3. __num_topics__: aantal te identificeren topic clusters. Dit is een van de belangrijkste parameters.
4. __update_every__: aantal chunks die verwerkt worden voordat de stap wordt gemaakt van E naar M (EM: Expectation Maximization).
5. __chunk_size__: aantal documenten die tegelijkertijd in geheugen geladen wordt.

Voor meer info: https://miningthedetails.com/blog/python/lda/GensimLDA/

Eerst importeren we de benodigde toolkits:

In [ ]:
import string
from gensim import corpora, models, similarities

We maken nu een dictionary van alle tokens in onze `textdict` variabele.

In [ ]:
# create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(textdict.values())

# remove extremes
dictionary.filter_extremes(no_below=2, no_above=0.8)

# convert dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in textdict.values()]

In [ ]:
n_topics = 11

# train the LDA model - TAKES A LONG TIME!
%time lda = models.LdaModel(corpus, num_topics=n_topics,id2word = dictionary,update_every=1,chunksize=100000,passes=50)

# save lda model
lda.save('./Output/lda '+ run_settings['datetime'] +'.model')

Laat de eerste 30 woorden zien van elk topic:

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=30)

# show topics
topics_matrix


In [ ]:
# output clusters to text file
with open("./Output/Run logfile "+run_settings["datetime"]+".txt", "a") as text_file:
    text_file.write("====================================\n")
    text_file.write("  LATENT DIRICHLET ALLOCATION model \n")
    text_file.write("====================================\n")
    text_file.write("Training set feature scores\n\n")
    for i in np.arange(0,len(topics_matrix)):
        text_file.write("Cluster {}\n".format(topics_matrix[i][0]))
        text_file.write(str(topics_matrix[i][1]))
        text_file.write("\n\n")
    text_file.write("\n")

### Document classificeren met LDA

Om nu nieuwe documenten, die niet gebruikt zijn bij het trainen van bovenstaand LDA model (en dus niet deel uitmaken van de standaard corpus), te classificeren met LDA, laden we eerst de desbetreffende documenten en tokenizen deze met de eerder gedefinieerde `processandtokenize` functie. Indien deze functie nog niet geladen is, run [deze](#processandtokenize) cell.

Daarna kunnen we de documenten invoeren in het LDA model. Hier komt vervolgens een score uit. In het geval een document scores behaalt binnen meerdere LDA clusters dan worden al deze scores gereturned. Als stopwords zijn verwijderd en de text 'gestemd' is voor de trainingsset doen we dat ook voor de validatieset.


In [ ]:
FilePath_val = './Rapporten selectie/validatie/'

# Get files in folder
FileNames_val = os.listdir(FilePath_val)

# Preallocate variables
textdict_val = {}
metadict_val = {}

for i in np.arange(np.shape(FileNames_val)[0]): # use this line for all files
    print "Processing ",i+1,'/',np.shape(FileNames_val)[0],': ', FileNames_val[i],'\n'
    
    try:
        data = pdfparser(FilePath_val,FileNames_val[i])
        
        # get PDF metadata
        pdfi = PdfFileReader(open("".join([FilePath_val,FileNames_val[i]]), "rb"))
        pdfi = pdfi.getDocumentInfo()
        metadict_val[FileNames_val[i]] = pdfi

        # split text
        data = np.array(re.split('\W+',data),dtype=str) # tokenize text
        
        # convert all data to lower case
        data = np.array([w.lower() for w in data])

        # filter out empty cells
        ind = data != ''
        data = data[ind]

        # filter out non-alpha strings
        ind = np.char.isalpha(data)
        data = data[ind]

        if data.size is not 0:
            # filter out strings with length = 1
            strlen = np.vectorize(len)
            ind = strlen(data) != 1
            data = data[ind]
        
            # add data to dict
            textdict_val[FileNames_val[i]] = list(data)

    except ValueError:
        print "Skipping ", FileNames_val[i]
        
# remove stopwords and stem text
if run_settings['stopwords_removed'] == True:
    removestopwords(textdict_val)
if run_settings['text_stemmed'] == True:
    stemtext(textdict_val)
        
# Create validation dictionary and corpus
dictionary_val = corpora.Dictionary(textdict_val.values())
corpus_val = [dictionary_val.doc2bow(text) for text in textdict_val.values()]

In [ ]:
# Classify texts and print results
for i in np.arange(0,np.shape(FileNames_val)[0]):
    topic_ids = sorted(lda[corpus_val[i]], key=lambda (index, score): -score)
    print 'LDA cluster      score'
    for j in topic_ids:
        print int(j[0]), '              ', j[1]
    print '\n'

In [ ]:
# output clusters to text file
with open("./Output/Run logfile "+run_settings["datetime"]+".txt", "a") as text_file:
    text_file.write("====================================\n")
    text_file.write("     LATENT DIRICHLET ALLOCATION    \n")
    text_file.write("====================================\n")
    text_file.write("Validation set classification cluster scores\n\n")
    for i in np.arange(0,num_clusters):
        topic_ids = sorted(lda[corpus_val[i]], key=lambda (index, score): -score)
        text_file.write(FileNames_val[i]+'\n')
        text_file.write("LDA cluster      score\n")
        for j in topic_ids:
            text_file.write(str(int(j[0])) + '                ' + str(j[1]) + "\n")
        text_file.write("\n")
    text_file.write("\n")

## Finaliseer logbestand

In [ ]:
# define function to add text to beginning of file
def line_prepender(filename, line):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)
        
# add title and settings info
head_settings = ""
for k,v in run_settings.items():
    head_settings = head_settings + str(k) + ': ' + str(v) + "\n"
header = run_settings['title'] + "\n\n" + "====================================\n" + \
         "             RUN SETTINGS             \n" + "====================================\n\n" + \
         head_settings + "\n\n"

line_prepender("./Output/Run logfile "+run_settings["datetime"]+".txt", header)

## Overig

### Workspace variabelen opslaan om later te gebruiken

In [ ]:
np.save('./Vars/lang.npy', lang)
np.save('./Vars/filepath.npy', FilePath)
np.save('./Vars/filenames.npy', FileNames)
np.save('./Vars/run_settings.npy', run_settings)
np.save('./Vars/textdict.npy', textdict)
np.save('./Vars/metadict.npy', metadict)

### Plot tijdlijn documenten

In [ ]:
import datetime

date = np.array([])

# convert metadict date field to date array
for i in np.arange(0,len(metadict)):
    year = metadict[metadict.keys()[i]]['/CreationDate'][2:6]
    month = metadict[metadict.keys()[i]]['/CreationDate'][6:8]
    day = metadict[metadict.keys()[i]]['/CreationDate'][8:10]

    # convert to python datetime object
    date = np.append(date,datetime.date(int(year), int(month), int(day)))
    

In [ ]:
# plot dates
import matplotlib.pyplot as plt
from matplotlib import dates as pltdates

fig = plt.figure()
ax = fig.gca()

# set plot properties
plt.title("Tijdlijn rapporten", fontsize=22)
fig.set_size_inches(20,20./60*len(textdict.keys()))
fig.gca().spines['left'].set_visible(False)
fig.gca().spines['right'].set_visible(False)
fig.gca().spines['top'].set_visible(False)
fig.gca().get_yaxis().set_ticks([]) # remove y ticks
fig.gca().axes.yaxis.set_ticklabels([]) # remove y labels
plt.grid(True,which='major',axis='x')

# plot data
xplot = np.arange(0,np.shape(textdict.keys())[0])
yind = np.argsort(date) # sort date for plotting
plt.scatter(date[yind], xplot, c=(75./256,151./256,182./256),s=100) 
for i in np.arange(0,np.shape(textdict.keys())[0]):
    plt.text(date[yind[i]]+datetime.timedelta(60), i, metadict.keys()[yind[i]], fontsize=8)
    
plt.xticks(np.arange(datetime.date(min(date).year,1,1),datetime.date(max(date).year+1,1,1),365),fontsize=16,rotation=70)
    
# make sure text fits in axis
plt.xlim(fig.gca().get_xlim()[0],fig.gca().get_xlim()[1]*1.003)

# show figure and save figure to working directory
plt.savefig('./Output/Rapport-datum_plot.pdf', dpi=80, orientation='landscape')
plt.show()


# IDEE: toekomstige implementaties

Tekst met keywords in pdf markeren, zie voorbeelden om locatie van het woord te extracten en de tekst in het PDF bestand te highlighten:

- https://stackoverflow.com/questions/7605577/read-highlight-save-pdf-programatically
- https://stackoverflow.com/questions/22898145/how-to-extract-text-and-text-coordinates-from-a-pdf-file/


Gebruik PCA of t-SNE bij clusteren


### LDA van sklearn



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# manipulate variable textdict to allow feeding into TfidfVectorizer function
alltext = [' '.join(text) for text in textdict.values()]

# define tokenizer
# dummy function: we split concatenated tokens at space
tf_tokenizer = lambda doc: [word for word in doc.split(' ')]

# define range of ngrams
ngrams = (3,15)

# define vectorizer -- using ngrams
#sklearn_tfidf = TfidfVectorizer(analyzer='char_wb',ngram_range=ngrams,norm='l2',min_df=1,max_df=0.8,
#                                use_idf=True,smooth_idf=False, sublinear_tf=True, tokenizer=tfidf_tokenizer)

sklearn_tf = CountVectorizer(max_df=0.95, min_df=2, tokenizer=tf_tokenizer, analyzer='char_wb',
                             ngram_range=ngrams)
tf_matrix = sklearn_tf.fit_transform(alltext)

#print tfidf_matrix.toarray()[0].tolist()
tf_feature_names = sklearn_tf.get_feature_names()

#print(np.shape(tfidf_matrix))
np.shape(tf_matrix)

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_topics = 10

nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5,
          init='nndsvd').fit(tfidf_matrix)

lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online',
                                learning_offset=50., random_state=0).fit(tf_matrix)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

no_top_words = 20
display_topics(nfm, terms, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

In [ ]:
FilePath_val = './Rapporten selectie/validatie/'

# Get files in folder
FileNames_val = os.listdir(FilePath_val)

# Preallocate variables
textdict_val = {}
metadict_val = {}

for i in np.arange(np.shape(FileNames_val)[0]): # use this line for all files
    print "Processing ",i+1,'/',np.shape(FileNames_val)[0],': ', FileNames_val[i],'\n'
    
    try:
        data = pdfparser(FilePath_val,FileNames_val[i])
        
        # get PDF metadata
        pdfi = PdfFileReader(open("".join([FilePath_val,FileNames_val[i]]), "rb"))
        pdfi = pdfi.getDocumentInfo()
        metadict_val[FileNames_val[i]] = pdfi

        # split text
        data = np.array(re.split('\W+',data),dtype=str) # tokenize text
        
        # convert all data to lower case
        data = np.array([w.lower() for w in data])

        # filter out empty cells
        ind = data != ''
        data = data[ind]

        # filter out non-alpha strings
        ind = np.char.isalpha(data)
        data = data[ind]

        if data.size is not 0:
            # filter out strings with length = 1
            strlen = np.vectorize(len)
            ind = strlen(data) != 1
            data = data[ind]
        
            # add data to dict
            textdict_val[FileNames_val[i]] = list(data)

    except ValueError:
        print "Skipping ", FileNames_val[i]
        
# remove stopwords and stem text
if run_settings['stopwords_removed'] == True:
    removestopwords(textdict_val)
if run_settings['text_stemmed'] == True:
    stemtext(textdict_val)
        
# Create validation dictionary and corpus
dictionary_val = corpora.Dictionary(textdict_val.values())
corpus_val = [dictionary_val.doc2bow(text) for text in textdict_val.values()]

In [ ]:
# classify unseen text
alltext_val = [' '.join(text) for text in textdict_val.values()]
tf_matrix_val = sklearn_tf.transform(alltext_val)
val_topics = lda.transform(tf_matrix_val)